# Step 1: Capture Voice and store it for later processing.
Enhanced version with better error handling and efficiency.

In [2]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
from collections import deque
from datetime import datetime

def record_until_silence(threshold=500, silence_duration=1.0, sample_rate=44100, max_duration=10):
    # Input validation
    if not all(isinstance(x, (int, float)) for x in [threshold, silence_duration, sample_rate, max_duration]):
        raise ValueError("All parameters must be numeric")
    if max_duration <= 0 or silence_duration <= 0:
        raise ValueError("Duration parameters must be positive")
    
    print("Initializing audio capture...")
    buffer = deque()  # More efficient than list for continuous appending
    silence_samples = int(silence_duration * sample_rate)
    silence_counter = 0
    peak_amplitude = 0
    
    def callback(indata, frames, time, status):
        nonlocal silence_counter, peak_amplitude
        if status:
            print(f'Status: {status}')
        
        current_amplitude = np.abs(indata).max()
        peak_amplitude = max(peak_amplitude, current_amplitude)
        buffer.append(indata.copy())
        
        # Check if the audio is below the threshold (silence)
        if current_amplitude < threshold:
            silence_counter += frames
        else:
            silence_counter = 0
            
        # Visual feedback of audio level
        if frames % (sample_rate // 2) == 0:  # Update twice per second
            print(f"Audio level: {'|' * int(current_amplitude/threshold)}")
            
        if silence_counter >= silence_samples:
            raise sd.CallbackStop
    
    try:
        with sd.InputStream(samplerate=sample_rate, channels=1, 
                          dtype=np.int16, callback=callback) as stream:
            print("Recording started... Speak now")
            sd.sleep(int(max_duration * 1000))
    except sd.CallbackStop:
        print("\nRecording stopped due to silence")
    except sd.CallbackAbort:
        print("\nRecording aborted due to device error")
    except Exception as e:
        print(f"\nError during recording: {str(e)}")
        return None, None

    if len(buffer) == 0:
        print("No audio recorded!")
        return None, None

    print(f"Peak amplitude detected: {peak_amplitude}")
    
    # Generate unique filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'recorded_audio_{timestamp}.wav'
    
    try:
        audio = np.concatenate(buffer, axis=0)
        wav.write(filename, sample_rate, audio)
        print(f"Audio saved as: {filename}")
        return audio, sample_rate
    except Exception as e:
        print(f"Error saving audio: {str(e)}")
        return None, None

# Record audio until silence is detected
audio_data, sr = record_until_silence()
if audio_data is not None:
    print(f"Successfully recorded {len(audio_data)/sr:.2f} seconds of audio")

Initializing audio capture...
Recording started... Speak now
Recording started... Speak now
Peak amplitude detected: 133
Audio saved as: recorded_audio_20250704_202052.wav
Successfully recorded 1.00 seconds of audio
Peak amplitude detected: 133
Audio saved as: recorded_audio_20250704_202052.wav
Successfully recorded 1.00 seconds of audio


# Step 2: Process audio and detect pattern and build memory support.